# Training Script

In [1]:
import os, object_detection
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

2023-03-09 12:33:57.069906: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Set up file structure and download chosen base model from Model Zoo

In [2]:
modelName = "640x640_model"
pretrainedModel = "ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8"
pretrainedModelURL = "http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz"

# Script sourced from Tensorflow: https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/tensorflow-1.14/training.html
tfRecordScript = "generate_tfrecord.py"
labelMap = "label_map.pbtxt"

In [17]:
filePaths = {
    "WORKSPACE": os.path.join("TF", "workspace"),
    "SCRIPTS": os.path.join("TF", "scripts"),
    "APIMODEL": os.path.join("TF", "models"),
    "ANNOTATIONS": os.path.join("TF", "workspace", "annotations"),
    "IMAGES": os.path.join("TF", "workspace", "images"),
    "PRETRAINED_MODELS": os.path.join("TF", "workspace", "pretrained_models"),
    'MODELS': os.path.join('Tensorflow', 'workspace','models'),
    'CHECKPOINTS': os.path.join('Tensorflow', 'workspace','models', modelName), 
}

In [18]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', modelName, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(filePaths['SCRIPTS'], tfRecordScript),
    'LABELMAP': os.path.join(filePaths['ANNOTATIONS'], labelMap)
}

In [19]:
for path in filePaths.values():
    !mkdir -p {path}

In [6]:
if not os.path.exists(os.path.join(filePaths['APIMODEL'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {filePaths['APIMODEL']}

Cloning into 'TF/models'...
remote: Enumerating objects: 81991, done.
remote: Counting objects: 100% (458/458), done.
remote: Compressing objects: 100% (204/204), done.
remote: Total 81991 (delta 293), reused 400 (delta 254), pack-reused 81533
Receiving objects: 100% (81991/81991), 596.38 MiB | 5.53 MiB/s, done.
Resolving deltas: 100% (58482/58482), done.


In [20]:
!wget {pretrainedModelURL}
!mv {pretrainedModel+'.tar.gz'} {filePaths['PRETRAINED_MODELS']}
!cd {filePaths['PRETRAINED_MODELS']} && tar -zxvf {pretrainedModel+'.tar.gz'}
!cp {os.path.join(filePaths['PRETRAINED_MODELS'], pretrainedModel, 'pipeline.config')} {os.path.join(filePaths['CHECKPOINTS'])}

--2023-03-09 12:57:47--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.200.16, 2a00:1450:4009:81d::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.200.16|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20518283 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.57M  34.4MB/s    in 0.6s    

2023-03-09 12:57:48 (34.4 MB/s) - ‘ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8.tar.gz’ saved [20518283/20518283]

ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_640x640_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_640x640_coco17_tp

# Create label mapping

In [9]:
objectLabels = [{'name':'resistor', 'id':1},
                {'name':'electrolytic_capacitor', 'id':2},
                {'name':'LED', 'id':3},
                {'name':'ceramic_capacitor', 'id':4},
                {'name':'IC', 'id':5}]

with open(files['LABELMAP'], 'w') as f:
    for label in objectLabels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

## Convert XML to TF record

In [10]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(filePaths['IMAGES'], 'train')} -l {files['LABELMAP']} -o {os.path.join(filePaths['ANNOTATIONS'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(filePaths['IMAGES'], 'test')} -l {files['LABELMAP']} -o {os.path.join(filePaths['ANNOTATIONS'], 'test.record')} 

Successfully created the TFRecord file: TF/workspace/annotations/train.record
Successfully created the TFRecord file: TF/workspace/annotations/test.record


## Set up model config file

In [24]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)
    
pipeline_config.model.ssd.num_classes = len(objectLabels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(filePaths['PRETRAINED_MODELS'], pretrainedModel, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(filePaths['ANNOTATIONS'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(filePaths['ANNOTATIONS'], 'test.record')]

config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text) 


# Training

In [33]:
import time

trainerScript = os.path.join(filePaths['APIMODEL'], 'research', 'object_detection', 'model_main_tf2.py')

trainCommand = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=5000".format(trainerScript, filePaths['CHECKPOINTS'],files['PIPELINE_CONFIG'])

# Trained using an Nvidia RTX 3060 GPU (12GB VRAM) for acceleration.
start = time.time()
!{trainCommand}
end = time.time()
elapsed = end - start
elapsedMins = elapsed / 60
print("Training time: {} seconds! ({} minutes)".format(round(elapsed, 2), round(elapsedMins, 0)))

2023-03-09 13:27:59.667871: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-09 13:28:01.224458: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-09 13:28:01.226028: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-09 13:28:01.226152: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
W0309 13:28:06.337535 139794484254528 deprecation.py:350] From /home/ben/.conda/envs/tf/lib/python3.10/site-packages/tensorflow/python/util/dispatch.py:1176: sample_distorted_bounding_box (from tensorflow.python.ops.image_ops_impl) is deprecated and will be removed in a future version.
Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
Instructions for updating:
Use `tf.cast` instead.
W0309 13:28:07.140437 139794484254528 deprecation.py:350] From /home/ben/.conda/envs/tf/lib/python3.10/site-packages/tensorflow/python/util/dispatch.py:1176: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
/home/ben/.conda/envs/tf/lib/python3.10/site-packages/keras/backend.py:451: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be re

# Evaluation

In [34]:
evalCommand = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(trainerScript, filePaths['CHECKPOINTS'],files['PIPELINE_CONFIG'], filePaths['CHECKPOINTS'])

!{evalCommand}

2023-03-09 13:28:40.171545: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-09 13:28:41.866017: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-09 13:28:41.867680: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-09 13:28:41.867791: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

Instructions for updating:
Use `tf.cast` instead.
W0309 13:28:44.843819 140694009034560 deprecation.py:350] From /home/ben/.conda/envs/tf/lib/python3.10/site-packages/tensorflow/python/util/dispatch.py:1176: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Waiting for new checkpoint at Tensorflow/workspace/models/640x640_model
I0309 13:28:46.100950 140694009034560 checkpoint_utils.py:140] Waiting for new checkpoint at Tensorflow/workspace/models/640x640_model
INFO:tensorflow:Found new checkpoint at Tensorflow/workspace/models/640x640_model/ckpt-6
I0309 13:28:46.101377 140694009034560 checkpoint_utils.py:149] Found new checkpoint at Tensorflow/workspace/models/640x640_model/ckpt-6
/home/ben/.conda/envs/tf/lib/python3.10/site-packages/keras/backend.py:451: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it,

^C
Traceback (most recent call last):
  File "/home/ben/Documents/Computer_Science/Hope/Core2/CircuitDetector/TF/models/research/object_detection/model_main_tf2.py", line 114, in <module>
    tf.compat.v1.app.run()
  File "/home/ben/.conda/envs/tf/lib/python3.10/site-packages/tensorflow/python/platform/app.py", line 36, in run
    _run(main=main, argv=argv, flags_parser=_parse_flags_tolerate_undef)
  File "/home/ben/.conda/envs/tf/lib/python3.10/site-packages/absl/app.py", line 308, in run
    _run_main(main, args)
  File "/home/ben/.conda/envs/tf/lib/python3.10/site-packages/absl/app.py", line 254, in _run_main
    sys.exit(main(argv))
  File "/home/ben/Documents/Computer_Science/Hope/Core2/CircuitDetector/TF/models/research/object_detection/model_main_tf2.py", line 81, in main
    model_lib_v2.eval_continuously(
  File "/home/ben/.conda/envs/tf/lib/python3.10/site-packages/object_detection/model_lib_v2.py", line 1135, in eval_continuously
    for latest_checkpoint in tf.train.checkpo